In [1]:
import pickle
import pandas as pd

In [16]:
year = 2023
month = 4

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/yellow_tripdata_{year:04d}-{month:02d}.parquet'

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [17]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df


In [18]:
df = read_data(input_file)
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [19]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [20]:
y_pred 

array([16.13130203, 32.26481598, 12.25557264, ..., 12.30118266,
       12.5560282 , 11.39399629], shape=(3199715,))

# Q1. Notebook

In [9]:
y_pred.std()

np.float64(6.247488852238703)

# Q2. Preparing the output

In [13]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [14]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [15]:
!ls -lh output

total 134072
-rw-r--r--@ 1 inigo_ocariz  staff    65M Jun 14 18:59 yellow_tripdata_2023-03.parquet


# Q5. Paremetrize the script

In [21]:
y_pred.mean()

np.float64(14.292282936862449)